# partition

In [ ]:
-- 非分区表修改为分区表
mysql > ALTER TABLE tablename BY RANGE COLUMNS(generate_time)
( 
PARTITION old_part VALUES LESS THAN ('2016-12-01 00:00:00'),
PARTITION p20161201 VALUES LESS THAN ('2016-12-02 00:00:00'),
PARTITION p20161202 VALUES LESS THAN ('2016-12-03 00:00:00'),
PARTITION p20161203 VALUES LESS THAN ('2016-12-04 00:00:00')
);

-- 增加分区
ALTER TABLE tablename ADD PARTITION (PARTITION p20161204 VALUES LESS THAN ('2016-12-05 00:00:00'));

-- 删除分区
ALTER TABLE tablename DROP PARTITION old_part;

# procedure

procedure可理解成MySQL中的UDF

In [ ]:
-- 查看procedure
show procedure status

-- 查看procedure详情
show create procedure procedure_name

-- 执行procedure
call procedure_name

In [ ]:
DELIMITER $$

DROP PROCEDURE IF EXISTS `proc_drop_add_partition`;
CREATE PROCEDURE `proc_drop_add_partition`(IN v_tablename VARCHAR(50))
BEGIN
START TRANSACTION;

/*
当前日期: 2016-12-14
创建Partition: p20161215 VALUES LESS THAN ('2016-12-16')
*/

-- 构建ALTER TABLE odps_frontend_request_internal_1min_result_sql DROP PARTITION p20161114
SET @partition_name=DATE_FORMAT(DATE_SUB(NOW(), INTERVAL 15 DAY), '%Y%m%d');
SET @t=CONCAT('ALTER TABLE ' ,v_tablename ,' DROP PARTITION p', @partition_name);

-- 执行ALTER
SELECT @t;
PREPARE stmt FROM @t;
EXECUTE stmt;
DEALLOCATE PREPARE stmt;

-- 构建ALTER TABLE odps_frontend_request_internal_1min_result_sql ADD PARTITION (PARTITION p20161215 VALUES LESS THAN ('2016-12-16 00:00:00'));
SET @partition_name=DATE_FORMAT(DATE_ADD(NOW(), INTERVAL 1 DAY), '%Y%m%d');
SET @partition_date=DATE_FORMAT(DATE_ADD(NOW(), INTERVAL 2 DAY), '%Y-%m-%d 00:00:00');
SET @s=CONCAT('ALTER TABLE ', v_tablename, ' ADD PARTITION (PARTITION p', @partition_name ,' VALUES LESS THAN (''', @partition_date, '''))');

-- 执行ALTER
SELECT @s;
PREPARE stmt FROM @s;
EXECUTE stmt;
DEALLOCATE PREPARE stmt;

-- 提交变更
COMMIT;
END;

$$
DELIMITER ;

# event

event可理解成MySQL中crontab

In [ ]:
-- 查看event
show events;

-- 查看event详情
show create event event_name

-- 查看定时调度是否打开
show VARIABLES LIKE 'event_scheduler'

-- 打开定时调度
SET GLOBAL event_scheduler = 1;

In [ ]:
DELIMITER $$

DROP EVENT IF EXISTS `event_name`;
CREATE EVENT `event_name`
ON SCHEDULE EVERY 1 DAY STARTS '2016-12-14 12:50:00'
ON COMPLETION PRESERVE
ENABLE
DO
BEGIN
CALL proc_drop_add_partition('table_name');
END;

$$
DELIMITER ;